In [1]:
import pickle
def mkPikle(filename, content):
    with open(filename, 'wb') as f:
        pickle.dump(content, f)
        f.close()
        
        
def rdPickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

Task 1: exclude stopwords. Identify **relative** frequency of lexical words before and after lockdown (data before 2020 and after 2020).

e.g. 1. the lexical word virus (tokens: viruses, virus, etc.) is used 20% per 10000 Tweets before lockdown and 30% after lockdown.<br>
e.g.2 mask is used is used 10% per 10000 Tweets before lockdown and 30% after lockdown.<br>
e.g.3 covid is used 0% before and 20 % after per 1000 Tweets <br>
e.g. 4 distance is used 20% per 10000 Tweets before lockdown and 30% after lockdown.<br>

Theoretical implication: the higher the change in frequency, the more probable is that a word arises in a new topic and/or has a new language use. 

Run n-gram or collocation analysis of the words with the highest frequency difference

Prediction: If a word appears with a new meaning, it will show differences in n-gram/collocations.

e.g. distance will appear with social as in social distancing after but not before the lockdown. 


In [2]:
import pandas as pd
#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
# lstTexts        = rdPickle('pickle/2017/lstTexts.pkl')
# df_2017         = rdPickle('pickle/2017/df_2017.pkl')
# countVector     = rdPickle('pickle/2017/countVector.pkl')
# countVectorizer = rdPickle('pickle/2017/countVectorizer.pkl')
# count_vect_df   = rdPickle('pickle/2017/count_vect_df.pkl')

In [4]:
lstTexts_old        = rdPickle('pickle/before_lock/lstTexts_old.pkl')
df_old              = rdPickle('pickle/before_lock/df_old.pkl')
countVector_old     = rdPickle('pickle/before_lock/countVector_old.pkl')
countVectorizer_old = rdPickle('pickle/before_lock/countVectorizer_old.pkl')
#count_vect_df_old   = rdPickle('pickle/before_lock/count_vect_df_old.pkl')
dicCV               = rdPickle('pickle/before_lock/dicCV.pkl')
df_old_summary      = rdPickle('pickle/before_lock/df_old_summary.pkl')

top_100             = rdPickle('pickle/before_lock/top_100.pkl' )
top_100_after       = rdPickle('pickle/2021-4/top_100.pkl')

lstTextToken        = rdPickle('pickle/before_lock/lstTextToken.pkl')
lstTextToken_org    = rdPickle('pickle/before_lock/lstTextToken_org.pkl')

Step 1 :Read the file

In [3]:
import pandas as pd
import os
df_2017 = pd.read_json(open( "/home/dell/workCode/Phd_works/topic_modeling/Covid_paper_work/examples_Twitter/en_geoTwitter17-10-22.json", 
                            "r", encoding="utf8"), lines=True)

In [4]:
df_2018 = pd.read_json(open( "/home/dell/workCode/Phd_works/topic_modeling/Covid_paper_work/examples_Twitter/en_geoTwitter18-01-20.json", 
                            "r", encoding="utf8"), lines=True)

In [5]:
df_old = pd.concat([df_2017,df_2018])

In [7]:
df_old['text'][2:4].tolist()

['Seattle WA Sun Oct 22nd PM Forecast: TONIGHT Chance Of T-Storm Lo 53 FRIDAY Showers Likely Hi 63',
 'Portobello mushroom steaks and yoga with goats. #coolerthanyou @ Hux Family Farm https://t.co/8VK0B4zbHy']

In [8]:
#preprocess(df_2017['text'][3:4].tolist()[0])

In [9]:
df_old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20552 entries, 0 to 11866
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 20552 non-null  datetime64[ns, UTC]
 1   id                         20552 non-null  int64              
 2   id_str                     20552 non-null  int64              
 3   text                       20552 non-null  object             
 4   source                     20552 non-null  object             
 5   truncated                  20552 non-null  bool               
 6   in_reply_to_status_id      224 non-null    float64            
 7   in_reply_to_status_id_str  224 non-null    float64            
 8   in_reply_to_user_id        353 non-null    float64            
 9   in_reply_to_user_id_str    353 non-null    float64            
 10  in_reply_to_screen_name    353 non-null    object             
 11  us

In [10]:
df_old.lang.value_counts()

en    20552
Name: lang, dtype: int64

In [11]:
df_old['place'][:1]

0    {'id': '65b4760a2b411e11', 'url': 'https://api.twitter.com/1.1/geo/id/65b4760a2b411e11.json', 'place_type': 'admin', 'name': 'New Jersey', 'full_name': 'New Jersey, USA', 'country_code': 'US', 'country': 'United States', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-75.563587, 38.788657], [-75.563587, 41.357424], [-73.88506, 41.357424], [-73.88506, 38.788657]]]}, 'attributes': {}}
Name: place, dtype: object

In [55]:
# Exploring the place field and country

#{'id': '65b4760a2b411e11', 'url': 'https://api.twitter.com/1.1/geo/id/65b4760a2b411e11.json', 
#'place_type': 'admin', 'name': 'New Jersey', 'full_name': 'New Jersey, USA', 'country_code': 'US', 
#'country': 'United States', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-75.563587, 38.788657], [-75.563587, 41.357424], [-73.88506, 41.357424], [-73.88506, 38.788657]]]}
#, 'attributes': {}}

df_old['place'].apply(pd.Series).country.value_counts()[:10]

United States                  14301
United Kingdom                  1018
Canada                           776
Republic of the Philippines      293
Brasil                           248
Nigeria                          209
Indonesia                        181
Malaysia                         166
Australia                        165
India                            149
Name: country, dtype: int64

In [13]:
df_old.timestamp_ms.dt.date.value_counts()

2018-01-20    11867
2017-10-23     8685
Name: timestamp_ms, dtype: int64


Pre-processing text data

Most of the text data are cleaned by following below steps.

    Remove punctuations
    Tokenization - Converting a sentence into list of words
    Remove stopwords
    Lammetization/stemming - Tranforming any form of a word to its root word



Another Cleanup which removes web links

In [131]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def replace_contractions(text):
    # creating an empty list
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
      expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    """Replace contractions in string of text"""
    return expanded_text

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

# function to remove special characters
def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def normalize1(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    #words = remove_stopwords(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)


In [15]:

def clean_text_v2(text):
    text = remove_URL(text)
    text = replace_contractions(text)
    text = re.sub('[0-9]+', '', text) # remove number
    
    text = remove_special_characters(text) # Special char
    text = remove_extra_whitespace_tabs(text) # Extra whitespace

    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Normalize
    words = normalize(tokens)

    return words

In [135]:
# Keeping the original text structure
def clean_text_v1(text):
    text = remove_URL(text)
    text = replace_contractions(text)
    text = re.sub('[0-9]+', '', text) # remove number
    text = remove_special_characters(text) # Special char
    text = remove_extra_whitespace_tabs(text) # Extra whitespace
    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Normalize
    words = normalize1(tokens)

    return words

In [45]:
# i =0 
# for txt in df_old['text'][14549:15000].tolist():
#     print(txt)
#     print(i, replace_contractions(txt))
#     i = i + 1

In [16]:
df_old['text_clean_v2'] = df_old['text'].apply(lambda x: clean_text_v2(x))

In [136]:
df_old['text_clean_v1'] = df_old['text'].apply(lambda x: clean_text_v1(x))

In [137]:
df_old[['text', 'text_clean_v1','text_clean_v2']][50:150]

,text,text_clean_v1,text_clean_v2
50,I'm at Ninoy Aquino International Airport (MNL) Terminal 3 - @miaagovph in Pasay City https://t.co/aqcizbgE7l,"[i, am, at, ninoy, aquino, international, airport, mnl, terminal, miaagovph, in, pasay, city]","[ninoy, aquino, international, airport, mnl, terminal, miaagovph, pasay, city]"
51,"When it’s this nice out, you invite a bunch of friends, the best… https://t.co/zF4jseykWu","[when, it, is, this, nice, out, you, invite, a, bunch, of, friends, the, best]","[nice, invite, bunch, friends, best]"
52,We wake up in the morning and piss excellence🍷 #developingweight… https://t.co/tVzc9hok09,"[we, wake, up, in, the, morning, and, piss, excellence, developingweight]","[wake, morning, piss, excellence, developingweight]"
53,"Just posted a photo @ Cabagan, Isabela https://t.co/oUntSOweo1","[just, posted, a, photo, cabagan, isabela]","[posted, photo, cabagan, isabela]"
54,"I'm at @TheMeltingPot in Houston, TX https://t.co/OWQJlzKtAP","[i, am, at, themeltingpot, in, houston, tx]","[themeltingpot, houston, tx]"
55,"Hello monday ,... \n\n#makeup #beauty #freshlook #natural #weddingday #party #bali… https://t.co/Pf4d6Umt2H","[hello, monday, makeup, beauty, freshlook, natural, weddingday, party, bali]","[hello, monday, makeup, beauty, freshlook, natural, weddingday, party, bali]"
56,“I been hurting you wouldn’t know the half; Imma snake and she forgot to move the grass”..… https://t.co/eppXO3rGUr,"[i, been, hurting, you, would, not, know, the, half, imma, snake, and, she, forgot, to, move, the, grass]","[hurting, would, know, half, imma, snake, forgot, move, grass]"
57,▶︎ $0 Down Auto Loans Available!◀︎\n✓ Choose Your Ride. \n✓ Drive As Soon As Today. \n✓ Bad… https://t.co/OmQDT4EugE,"[down, auto, loans, available, choose, your, ride, drive, as, soon, as, today, bad]","[auto, loans, available, choose, ride, drive, soon, today, bad]"
58,The food is good... #happypride #pride #happypride2017 #yummyyum #yummy #yummyformytummy… https://t.co/B3s8uAERzm,"[the, food, is, good, happypride, pride, happypride, yummyyum, yummy, yummyformytummy]","[food, good, happypride, pride, happypride, yummyyum, yummy, yummyformytummy]"
59,Way to go you Derby's! Justin Williams AA Tournament Champs! @… https://t.co/Sfdgys6vja,"[way, to, go, you, derby, s, justin, williams, aa, tournament, champs]","[way, go, derby, justin, williams, aa, tournament, champs]"


Passing through count vectorization

In [20]:
lstTexts_old = df_old['text_clean_v2'].apply(lambda x: " ".join(x)).tolist()
lstTexts_old

['nice see water back lake camp roosevelt',
 'screem fest canobie lake park',
 'seattle wa sun oct nd pm forecast tonight chance tstorm lo friday showers likely hi',
 'portobello mushroom steaks yoga goats coolerthanyou hux family farm',
 'old enough decide eat birthday cake breakfast hayes valley san francisco',
 'karval co',
 'um happened got picked gate lax coming plane escorted',
 'getting ready watch falcons pictures melissa rivalries',
 'know new york upperwestside uws food market sundayfunday newyork',
 'stevie wonder circuit americas',
 'posted photo vancouver washington',
 'thejacksons dusse blacklove fashiongeekzo x t_abbey love lombard',
 'hackensack clinton street always halloween halloweendecorations hackensack',
 'turn timberwolves game soon possible ur watching repost timberwolves basket',
 'reading shaman shamanichealing shamanicjourney lake gaston',
 'broken clouds gt clouds temperature c gt c',
 'happy working day kansai felt sdn bhd',
 'cmx vip cinema experience miam

In [139]:
lstTextToken = df_old['text_clean_v2'].tolist()
lstTextToken = list(itertools.chain.from_iterable(lstTextToken))
lstTextToken[:1],len(lstTextToken)

(['nice'], 171773)

In [154]:
# Keeping the original text
lstTextToken_org = df_old['text_clean_v1'].tolist()
lstTextToken_org = list(itertools.chain.from_iterable(lstTextToken_org))
lstTextToken_org[:10], len(lstTextToken_org)

(['it', 'is', 'nice', 'to', 'see', 'some', 'water', 'back', 'in', 'the'],
 252965)

In [168]:
# Saving the pickle
mkPikle('pickle/before_lock/lstTextToken.pkl', lstTextToken)
mkPikle('pickle/before_lock/lstTextToken_org.pkl', lstTextToken_org)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
countVectorizer_old = CountVectorizer() 
countVector_old = countVectorizer_old.fit_transform(lstTexts_old)

In [28]:
countVector_old

<20552x30303 sparse matrix of type '<class 'numpy.int64'>'
	with 165270 stored elements in Compressed Sparse Row format>

In [29]:
columns=countVectorizer_old.get_feature_names()

In [54]:
len(columns)

30303

Working with Count vectors

In [32]:
count_vect_df_old = pd.DataFrame(countVector_old.toarray(), columns=columns)

In [33]:
count_vect_df_old.columns

Index(['__', '____', '______', '__________', '___________', '______________',
       '______________________________', '________________________________',
       '___________________________________',
       '_____________________________________',
       ...
       'zumas', 'zumba', 'zumbafamily', 'zumbaladies', 'zumbalife', 'zunck',
       'zurich', 'zurymaar', 'zyrus', 'zzz'],
      dtype='object', length=30303)

Saving to pickle

In [24]:
mkPikle('pickle/before_lock/df_old.pkl', df_old)
mkPikle('pickle/before_lock/lstTexts_old.pkl', lstTexts_old)

mkPikle('pickle/before_lock/countVector_old.pkl', countVector_old)
mkPikle('pickle/before_lock/countVectorizer_old.pkl', countVectorizer_old)

In [45]:
mkPikle('pickle/before_lock/dicCV.pkl', dicCV)
mkPikle('pickle/before_lock/df_old_summary.pkl', df_old_summary)

In [73]:
mkPikle('pickle/before_lock/top_100.pkl', top_100)

In [ ]:
#mkPikle('pickle/before_lock/count_vect_df_old.pkl', count_vect_df_old)

In [37]:
import numpy as np
def getTermValue(wrd):
    lstCV = count_vect_df_old[wrd].tolist()
    #print(lstCV)
    total_occur = sum(lstCV)
    number_doc  = np.count_nonzero(lstCV)
    return total_occur, number_doc, len(lstCV)
    #return( total_occur * number_doc / len(lstCV))

In [38]:
getTermValue('work')

(1590, 1585, 20552)

In [39]:
import pandas as pd
dicCV = {}
#create dataframe
df_old_summary = pd.DataFrame({})

for colVal in columns:
    print(colVal) 
    total_occur, number_doc, total = getTermValue(colVal)
    rf = total_occur/ total
    #Add in the dictionary
    dicVal = (total_occur * number_doc) / total
    dicCV[colVal] = dicVal
    # add row
    new_row = {'word':colVal, 'total_occur':total_occur, 'number_docs':number_doc, 'rf': rf, 'total':total, 'dicVal': dicVal}
    df_old_summary = df_old_summary.append(new_row, ignore_index=True)


__
____
______
__________
___________
______________
______________________________
________________________________
___________________________________
_____________________________________
____mmd
__barbiiee
__carmelaaaa
__chp__
__glittergirlz
__king__jabo__
__parizat
_almutairii
_amirulabdullah
_ammyyy
_anaaelling
_babydime
_blackrosekingz
_blacksunempire
_clairestreet
_cutloose
_dippstick_
_harbi
_how_
_ikarus
_john_ryder_
_joshsnyder
_kaitlynd
_katie_joy_
_lingerie_paris
_mayback
_oneclick
_overwatch_
_pereiraisa
_polymetis_
_rebeccanewman_
_sam_laves_
_santiagoblandn
_shtnonm
_sisters
_stefanseyes
_taiiiii
_taymontana_
_theconjure_
_ulysses_
_usil
_weed_universe
a_cukurova
a_limitlessdance
a_natin
aa
aaa
aaaaa
aabddullah
aacres
aafmidlands
aai
aakandary
aalam
aalrshedie
aamaadmiparty
aamp
aannaabarreiro
aanrader
aapiseverywhere
aapkefightermlas
aarde
aare
aarhus
aaron
aaronfranklin
aaronfukkinlee
ab
aba
abalone
abandoned
abant
abarare
abbaz
abbeybrewingmia
abbeylynnphotography
ab

amount
amounts
amour
amp
ampang
ampcleveland
amphitheater
amphitheatre
ample
ampm
amrap
ams
ams__
amseena
amstelstation
amsterdam
amsterdambyday
amsterdamdanceevent
amtrak
amusing
amway
amy
amyallenwhitfield
amystephens
ana
anacosta
anagarden
anaheim
anaheimsanta
analog
analysis
analyst
analystcoordinator
analytic
analytics
anappleaday
anashe
anatomy
anawfulsound
anaya
anbadloville
anc
anchor
anchorage
anchovies
ancient
ancoragens
andalous
andalucia
andalusia
andaya
andel
andela
anderson
andersonberinja
andfollow
andheri
andi
andien_dintchy
anditaputri
andmochis
andor
andover
andovergvsquash
andrade
andre
andrea
andrecarriell
andreduarte
andrew
andreward
andrewkolar
andrewlincoln
andrews
andrewwayne
andries
android
androidpaid
andromedainitiative
andy
andyg
andymineo
andys
andyvargasmusic
andyyytok
aneliesiepr
anesthesia
anesthesiologist
anesthesiology
anew
anfield
ang
angel
angelaismonroe
angelanicolaou_
angelarye
angeles
angelfilter
angelic
angelike
angelo
angeloni
angels
anger
anger

aztecs
azura
azure
azusa
azwx
azz
ba
baaaack
baahubali
baba
babareshi
babbling
babe
babelaufenberg
babes
babesofinstagram
babi
babies
bablo
baby
babygap
babygenius
babygirl
babygotback
babyjade
babyjax
babylisspro
babylon
babymandap
babys
babyshark
babyshower
babysitting
babyspam
babyswims
babywantscandy
babyyyy
bac
bacardiblack
bacardimusiccds
bacaro
bacchanal
bacchus
bachata
bachour
back
backdrop
backed
background
backgrounds
backing
backkk
backkkk
backlick
backpack
backpacker
backs
backside
backstreetboys
backthatpassup
backtoback
backtoreality
backtothefuture
backtweets
backup
backwork
backyard
bacliff
bacolor
bacon
bacoor
bacutia
bad
badass
badblackbarbie
badboy
badboyforlife
badcompany
badddddd
baddestdjrolex
baden
badest
badge
badger
badgers
badgyaldomi
badhairday
badinfluenceandbanter
badling
badly
badminton
badowl
badowlcoffee
badraah
badrabbits
badshah
badung
bae
baebangor
baedimples
bag
bagan
bagel
bagelry
bagels
baggage
bagh
bagladies
bags
baguio
bahamas
bahamasmusic
bahiab

bhimbyknows
bhimtodeblodclateorld
bhk
bhldn
biancaafreire
bias
bibb
bibia
bible
biceps
bichpoo
bici
bicol
bicycle
bid
biddeford
bidding
bidmc
bidwell
bieber
bier
biersch
big
bigaseidel
bigbankdte
bigbear
bigbeerlist
bigbendbrewing
bigbiceps
bigchefscafe
bigconference
bigcurls
bigdaddy
bigdaddygunscom
bigdru
bigeast
bigeasy
biggarankinstreetngreet
bigger
biggest
biggie
biggrass
biggsdiner
biginjapan
biglangbaguio
biglittlereveal
bigmac
bigpress
bigracks
bigrig
bigrivercrossing
bigsky
bigskycountry
bigspender
bigspring
bigstonegap
bigstrongfast
bigtex
bigummsneworleans
bigup
bigworldtour
bih
bihon
bike
bikelife
biker
bikes
bikesandnanos
bikeschool
biketoberfest
biketoberfestrally
biking
bikini
bikinibeachcv
bikinis
bila
bilanaktinggi
bilbao
bildnis
biles
bilingual
bill
billboard
billdixonish
billede
billericay
billguillotina
billiards
billing
billings
billion
billionaires
bills
billsforever
billsmafia
billson
billstailgate
billund
billy
billyelliot
billyidol
billyjoel
billyque_b
biloxi
b

brewtopiaevents
bri
brian
briana
briankendrick
brianne
briansfinest_hd_shmc
briansimpson
bribe
brice
bricescipio
brick
brickell
bridal
bridalhairstylenofilter
bridalsession
bride
brides
bridesandbouquetsnc
bridesmaid
bridge
bridgehampton
bridgeport
bridgers
bridges
bridgeton
bridgetown
bridgeville
bridgewater
bridle
brief
briefings
briggs
bright
brighten
brightening
brighter
brightest
brighton
brilliant
brindlepug
brined
bring
bringatob
bringin
bringing
bringonsummer
brings
brisbane
briscoemanor
brisk
brisket
bristol
britain
britains
british
britishcolumbia
britspubmpls
brittany
brittanyspaniel
brittanyspanielsofinsta
brittle
brixxpizza
bro
broad
broadalbin
broadcast
broadcasting
broadview
broadway
broadwayinchicago
broadwaynightclub
broadwaysd
broadwayseventh
broccoli
broccolicity
brock
brockbank
brockgill
brockley
brockport
broda
brodie
brognoli
broing
broke
broken
brokenarrow
brokenback
brokenmindshard
broker
brokerage
bromeliad
bromeliadguzmania
bromont
bromore
bromwich
broncos
bro

...

sweatpink
sweatshirt
sweaty
sweatysaturday
sweden
swedish
swedishart
swedishjobs
swedishphotography
sweeep_
sweeney
sweep
sweeps
sweepstakes
sweet
sweetbindi
sweetboy
sweeter
sweetest
sweetfrog
sweetgreens
sweetheart
sweetheat
sweethome
sweetie
sweeties
sweetland
sweetly
sweetnesscakes
sweettreat
sweetvicky
sweetz
swell
swells
swerve_fitness
sweta
swfc
swift
swim
swimava
swimmer
swimming
swimsuit
swimwear
swindon
swing
swinging
swingingbridgebrewing
swingthatcat
swip
swipe
swipeleft
swirl
swiss
swissmountainview
swisspub
swisssuperman
switch
switzerland
switzerlandootd
swkts
swmrrupters
swmrs
swole
swollen
swolliets
swollios
swordartonline
swordartonlinecosplay
swordartonlineordinalscale
swprophets
swung
swvawx
swx
syafinaz
syarii
syazantsev
sycamore
sydenham
sydney
sydneypvd
syfi
sykeee
sylacauga
sylly
sylvan
sylvania
sylvester
symmetry
sympathy
syncing
synergy
synergygymsnyc
synergym
syngenta
synthesizer
synths
syowa
syr
syracuse
syrah
syria
sys
system
systems
syyounger
sz
szn
t_abbe

thetravellingbanana
thetraveltester
thetrumpreport
theuk
theusesofanger
thevalueseries
thevervepipe
thevigaz
thevintagephoenix
thewalk
thewalkingdead
thewalkingdeadfamily
thewayisalwaysforward
theweekend
theweekendhasarrived
theweeknd
thewhitehairchick
thewhitehouse
thewindycity
thewolfbham
thewoodlands
thewritingforum
theyankeecandle
theymightbegiants
thezonecast
thick
thickcurlgurl
thief
thier
thieves
thika
thimphu
thin
thing
things
thingsisee
thingsyouseeonthebike
think
thinkblue
thinkers
thinkgreen
thinkin
thinking
thinkit
thinks
thinmanbrewery
thinner
thinningtheherd
thionville
third
thirdstreetsouthfarmersmarket
thirsty
thirties
thirty
thirtyysevenx
thirumangalam
thiscity
thisdayinmetal
thishereland
thisis
thisisbreadbar
thisislove
thisismyla
thisismymoment
thisisourselfieface
thisisphilly
thisistwinsterritory
thisiswhatafeministlookslike
thisiswhatdemocracylookslike
thisiswhatfeminismlookslike
thisorder
thisteam
thistimelastmonth
tho
thogston
thol
thomas
thomascreekbeer
thomasfi

tumwater
tuna
tundeednut
tune
tuned
tunein
tunes
tung
tunic
tuning
tuninho
tunisia
tunkhannock
tunku
tunnel
tupelo
tupid
turbo
turcomp
turf
turfmoor
turkey
turkeymeatballs
turkishbacked
turkishvan
turkiye
turks
turkshutter
turku
turmeric
turn
turnaround
turned
turner
turning
turnodetarde
turnout
turnpike
turns
turnstl
turnt
turntable
turntablekitchen
turntables
turntablism
turntup
turodelhome
turpal
turquoise
turrondevodka
turtlediving
turtleman
turtleneck
turtles
turtlestrydive
turveydrop
tuscaloosa
tuscon
tussarsilk
tussauds
tustin
tustinflag
tutorial
tv
tver
tvjohnny
tvnews
tvontheradio
tvseries
tvtower
tvvikings
tw
twale
twd
twdisback
twdnight
twdnite
twdseason
twdsunday
tweedles
tweegram
tweet
tweeted
tweets
tweezers
twelve
twenties
twenty
twentyeighteen
twentysixthchapter
twice
twilight
twilightbanquethall
twin
twinkletwinklelittlestar
twinpeakslbv
twins
twinsburg
twinsfest
twintower
twist
twisted
twistedtea
twists
twitch
twitchcon
twitter
twitterblades
two
twoaday
twobirdsbrewin

vwry
vybzzzzzz
wa
waaaaaay
wack
waconia
wade
wadi
wadley
waffeln
waffle
waffles
wafflin
wagner
wagon
wagonlit
wagons
wags
wah
wahpeton
wahran
wai
waif
waikato
waikiki
waikikibeach
waikoloavillage
wailea
waimanalo
wainscoting
waipahu
waist
wait
waiting
waitress
waitresses
waits
waitstaffserver
wake
wakeandmake
wakeboarding
wakefield
wakepark
wakes
waking
waldalalii
waldorf
wales
walesinautumn
walgreens
walidfrenchmiami
walk
walkaway
walked
walker
walkhearing
walkies
walkin
walking
walkingdead
walkingdead_amc
walkingdogs
walkinwithkelley
walkon
walks
walktheline
walkup
walkway
wall
wallawalla
wallball
wallig
wallmiami
wallpaper
walls
wallsend
wallstreet
wally
walmart
walnut
walnutcreek
walnuts
walpole
walser
walsh
walt
waltdisneystudios
waltdisneyworld
waltdisneyworldresort
walter
waltham
waltonverona
wambuimukenyi
wan
wander
wanderbeyond_
wandered
wanderer
wandering
wanderings
wanderlust
wanderlusticeland
wangsa
wangz
wanjavalerie
wannaone
wannaoneinkl
wannasorn
want
wanted
wanting
want

womensmarchtn
womensmarchto
womensmarchtoronto
womensmarhnyc
womensrights
womensrightsarehumanrights
womensservices
womenworkbettertogether
wonder
wonderful
wonderfully
wondergirl
wondering
wonderland
wonders
wonderwoman
woo
wood
woodberrycoffee
woodbine
woodbridge
woodbridgetownship
woodburn
woodburnbrewery
woodbury
woodclifflake
woodcock
wooden
woodford
woodland
woodlands
woodley
woodpile
woodriver
woodruffpark
woods
woodside
woodson
woodstock
woodward
woodworking
woody
woof
woohoneychild
woohoo
wool
woolbright
woolliment
woonsocket
wooo
woooo
woooooooooooooooooooooo
woooooowwwww
woops
wooster
woot
wootton
worcester
word
wordbearers
wordcampabq
worden
wordpress
words
wordstoliveby
wordswithfriends
wore
work
workandplay
workbobs
workbook
workday
workdays
worked
worker
workers
workflow
workforce
workgrindprogress
workhard
workin
working
workinghardorhardlyworking
workingmoms
worklife
workman
workout
workouts
works
worksbagelthat
workship
workshop
workshops
worksop
workspace
worktrip
wo

In [58]:
sorted(dicCV.items(), key=lambda kv: kv[1],reverse=True)

[('job', 1293.2643051771117),
 ('hiring', 1045.5356169715842),
 ('careerarc', 458.8867750097314),
 ('latest', 198.93363176333204),
 ('work', 122.62310237446478),
 ('opening', 118.86770144024912),
 ('click', 91.32444530945894),
 ('great', 89.06967691708836),
 ('see', 70.533281432464),
 ('want', 57.12193460490463),
 ('jobs', 53.95139159205917),
 ('anyone', 53.49124172829895),
 ('recommend', 51.820942000778516),
 ('fit', 51.3699883223044),
 ('ca', 46.11210587777345),
 ('apply', 41.36259244842351),
 ('amp', 32.26936551187232),
 ('team', 29.907162319968858),
 ('wind', 26.285714285714285),
 ('today', 25.15142078629817),
 ('new', 22.335052549630205),
 ('photo', 21.1308388478007),
 ('humidity', 20.747810432074736),
 ('join', 18.28381666017906),
 ('could', 17.458203581159985),
 ('looking', 16.73715453483846),
 ('check', 16.42472752043597),
 ('view', 16.424532892175943),
 ('retail', 16.413779680809654),
 ('mph', 13.89207862981705),
 ('interested', 12.955235500194629),
 ('might', 12.8050311405216

In [72]:
lst_100 = sorted(dicCV.items(), key=lambda kv: kv[1],reverse=True)[:100]
# Creating a top 100 set of words 
top_100 = list(map(lambda x: x[0], lst_100))
top_100[:5]

['job', 'hiring', 'careerarc', 'latest', 'work']

In [76]:
# Reading 2021 top 100 and finding the common
top_100_after = rdPickle('pickle/2021-4/top_100.pkl')

In [80]:
intersection_set = set(top_100).intersection(top_100_after)
#list(intersection_set), len(list(intersection_set))

In [86]:
list(set(top_100_after) ^ intersection_set)

['quarantine',
 'portland',
 'stay',
 'easter',
 'florida',
 'km',
 'texas',
 'dinner',
 'north',
 'little',
 'covid',
 'incident',
 'live',
 'let',
 'street',
 'thank',
 'know',
 'link',
 'people',
 'open',
 'would',
 'video',
 'going',
 'free',
 'co',
 'april',
 'support',
 'follow',
 'san',
 'untappd',
 'family',
 'every',
 'way',
 'los',
 'look',
 'days',
 'make',
 'made',
 'still',
 'ft',
 'week',
 'beach',
 'reports',
 'bio']

In [41]:
df_old_summary[(df_old_summary.total_occur>2) & (df_old_summary.number_docs>3) 
                & (df_old_summary.word.str.len()>2) & (df_old_summary.dicVal>1)]

,word,total_occur,number_docs,rf,total,dicVal
857,amazing,153.0,148.0,0.007445,20552.0,1.101791
924,amp,829.0,800.0,0.040337,20552.0,32.269366
1124,anyone,1049.0,1048.0,0.051041,20552.0,53.491242
1199,apply,922.0,922.0,0.044862,20552.0,41.362592
1519,associate,145.0,144.0,0.007055,20552.0,1.015960
1852,back,222.0,214.0,0.010802,20552.0,2.311600
2100,barometer,172.0,172.0,0.008369,20552.0,1.439471
2281,beach,153.0,143.0,0.007445,20552.0,1.064568
2348,beautiful,188.0,179.0,0.009148,20552.0,1.637408
2620,best,190.0,184.0,0.009245,20552.0,1.701051


In [42]:
df_old_summary[(df_old_summary.total_occur>2) & (df_old_summary.number_docs>3) 
                & (df_old_summary.word.str.len()>2) & (df_old_summary.dicVal>1)].count()

word           97
total_occur    97
number_docs    97
rf             97
total          97
dicVal         97
dtype: int64

In [43]:
#(total_occur / total) = (rf * df)

In [88]:
df_old_summary[(df_old_summary.total_occur>2) & (df_old_summary.number_docs>3) 
                 & (df_old_summary.dicVal>1)].sort_values(by=['total_occur'], ascending=False)[:5]


,word,total_occur,number_docs,rf,total,dicVal
13962,job,5159.0,5152.0,0.251022,20552.0,1293.264305
12262,hiring,4639.0,4632.0,0.225720,20552.0,1045.535617
4405,careerarc,3071.0,3071.0,0.149426,20552.0,458.886775
15106,latest,2022.0,2022.0,0.098385,20552.0,198.933632
29775,work,1590.0,1585.0,0.077365,20552.0,122.623102


In [52]:
df_old_summary[df_old_summary.word.isin(
    ['covid','virus','quarantine','lockdown','mask',
     'distance','corona'])]

,word,total_occur,number_docs,rf,total,dicVal
5137,china,8.0,7.0,0.000389,20552.0,0.002725
6142,corona,5.0,4.0,0.000243,20552.0,0.000973
7557,distance,3.0,3.0,0.000146,20552.0,0.000438
16626,mask,5.0,5.0,0.000243,20552.0,0.001216
28701,virus,1.0,1.0,0.000049,20552.0,0.000049


In [14]:
import nltk
from nltk.collocations import * 

def getColocation(text, cls, typeText):
    if typeText=="org":
        lstText = lstTextToken_org
    else:
        lstText = lstTextToken
    # Ngrams with 'creature' as a member
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    trigram_measures = nltk.collocations.TrigramAssocMeasures()# Ngrams with 'creature' as a member
    
    ## Bigrams
    finder2 = BigramCollocationFinder.from_words(lstText)
    finder3 = TrigramCollocationFinder.from_words(lstTextToken)


    if cls==2:
        cls = bigram_measures
        finder = finder2
    else:
        cls = trigram_measures
        finder = finder3
        
    creature_filter = lambda *w: text not in w

    # only bigrams that appear 3+ times
    #finder.apply_freq_filter(3)

    # only bigrams that contain 'creature'
    finder.apply_ngram_filter(creature_filter)

    # return the 10 n-grams with the highest PMI
    print (finder.nbest(cls.likelihood_ratio, 10))
    for i in finder.score_ngrams(cls.likelihood_ratio):
        print (i)    

In [23]:
getColocation("quarantine",2,"org")

[]


In [24]:
getColocation("quarantine",3,"org")

[]


In [8]:
# ## Trigrams
# trigram_measures = nltk.collocations.TrigramAssocMeasures()# Ngrams with 'creature' as a member
# cls = trigram_measures
# creature_filter = lambda *w: 'corona' not in w
# finder = TrigramCollocationFinder.from_words(lstTextToken)
# # only trigrams that appear 3+ times
# #finder.apply_freq_filter(3)
# # only trigrams that contain 'creature'
# finder.apply_ngram_filter(creature_filter)
# # return the 10 n-grams with the highest PMI
# # print (finder.nbest(trigram_measures.likelihood_ratio, 10))
# for i in finder.score_ngrams(cls.likelihood_ratio):
#     print (i)